# Initializing and Restoring EMI-RNN Graphs

The *EMI-RNN* implementation supports four forms of initialization/restoring:
1. An entirely new graph can be constructed with randomly initialized weights.
2. A saved graph can be loaded into the current `EMI_Driver`.
2. An entirely new graph can be constructed with weights initialized from a saved graph. This behavior is essentially restoration of a saved graph.
3. (*Experimental*) Initializing/Restoring using numpy matrices. 

All three methods will be illustrated in this notebook. This notebook uses the Peliculas dataset and builds on the [EMI LSTM example.ipynb](https://github.com/adithyaprem/Real-Time-Crime-Surveillance-Using-Edge-Devices/blob/adi/Peliculas/Peliculas%2012_3%20EMI-LSTM%20Implementation.ipynb)

In [ ]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# To include edgeml in python path
sys.path.insert(0, '../../')
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# MI-RNN and EMI-RNN imports
from rnn import EMI_DataPipeline
from rnn import EMI_FastGRNN
from emirnnTrainer import EMI_Trainer, EMI_Driver
import utils

Let us set up some network parameters for the computation graph.

In [ ]:
MODEL_GRNN = 'model-fastgrnn'

In [ ]:
NUM_HIDDEN = 128
NUM_TIMESTEPS = 12
NUM_FEATS = 540
FORGET_BIAS = 1.0
NUM_OUTPUT = 2
USE_DROPOUT = 1

KEEP_PROB = 0.9
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"
WRANK = 5
URANK = 6
PREFETCH_NUM = 5
BATCH_SIZE = 32
NUM_EPOCHS = 3
NUM_ITER = 4
NUM_ROUNDS = 10

# # A staging direcory to store models
MODEL_PREFIX = '<model-prefix>'

In [ ]:
def setEnvironment(params, folder, location):
    global NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, FORGET_BIAS, NUM_OUTPUT, USE_DROPOUT, KEEP_PROB, PREFETCH_NUM, BATCH_SIZE, NUM_EPOCHS, NUM_ITER, NUM_ROUNDS, MODEL_PREFIX, DATA_PREFIX, UPDATE_NL, GATE_NL, WRANK, URANK
    NUM_HIDDEN = params['NUM_HIDDEN']
    NUM_TIMESTEPS = params['NUM_TIMESTEPS']
    NUM_FEATS = params['NUM_FEATS']
    FORGET_BIAS = params['FORGET_BIAS']
    NUM_OUTPUT = params['NUM_OUTPUT']
    USE_DROPOUT = True if params['USE_DROPOUT'] == 1 else False
    KEEP_PROB = params['KEEP_PROB']
    PREFETCH_NUM = params['PREFETCH_NUM']
    BATCH_SIZE = params['BATCH_SIZE']
    NUM_EPOCHS = params['NUM_EPOCHS']
    NUM_ITER = params['NUM_ITER']
    NUM_ROUNDS = params['NUM_ROUNDS']
#     MODEL_PREFIX = os.path.join(folder, MODEL_GRNN)
    MODEL_PREFIX = "/home//UCF11_fastgrnn"
    UPDATE_NL = params['UPDATE_NL']
    WRANK = params['WRANK']
    URANK = params['URANK']
    GATE_NL = params['GATE_NL']
    DATA_PREFIX = os.path.join(folder, location) + '/'

# Loading Data

In [ ]:
params = {
    "NUM_HIDDEN" : 64,
    "NUM_TIMESTEPS" : 12,
    "NUM_FEATS" : 540,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 2,
    "USE_DROPOUT" : 0, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.9,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "WRANK" : 5,
    "URANK" : 6,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 3,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
}
setEnvironment(params, './Peliculas', 'RAW')

In [ ]:
# Loading the data
DATA_PREFIX = '/path/of/feature/set'
# x_train, y_train = np.load(DATA_PREFIX + 'x_train.npy'), np.load(DATA_PREFIX + 'y_train.npy')
x_test, y_test = np.load(DATA_PREFIX + 'x_test.npy'), np.load(DATA_PREFIX + 'y_test.npy')
x_val, y_val = np.load(DATA_PREFIX + 'x_val.npy'), np.load(DATA_PREFIX + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
# BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_test.shape[1]
# print("x_train shape is:", x_train.shape)
# print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

In [ ]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True
    
def addExtendedAssignOps(self, graph, W_val=None, B_val=None):
    W1 = graph.get_tensor_by_name('W1:0')
    B1 = graph.get_tensor_by_name('B1:0')
    W1_op = tf.assign(W1, W_val)
    B1_op = tf.assign(B1, B_val)
    self.assignOps.extend([W1_op, B1_op])

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob, **kwargs):
    feedDict = {self._emiGraph.keep_prob: keep_prob}
    return feedDict
    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
EMI_FastGRNN.addExtendedAssignOps = addExtendedAssignOps

def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index


if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

## 1. Initializing a New Computation Graph

In the most common use cases, a new EMI-RNN graph would be created and trained

In [ ]:
tf.reset_default_graph()

inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

# Construct the graph
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)


Lets initialize a new session with this graph and train a model. The saved model will be used later for restoring.

In [ ]:
emiDriver.initializeSession(g1)
# y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
#                                       y_train=y_train, bag_train=BAG_TRAIN,
#                                       x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
#                                       numIter=NUM_ITER, keep_prob=KEEP_PROB,
#                                       numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
#                                       numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
#                                       fracEMI=0.5, updatePolicy='top-k', k=1)

As the output above indicates, the last restored model is `/tmp/model-lstm-1001`. That is, with `MODEL_PREFIX = '/tmp/model-lstm'` and `GLOBAL_STEP=1001`.

In [ ]:
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index
emiDriver.initializeSession(g1)

In [ ]:
%%time
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

## 2. Loading a Saved Graph into EMI-Driver

We will reset the computation graph, load a saved graph into the current `EMI_Driver` and verify its outputs.

In [ ]:
tf.reset_default_graph()

emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX, 1030)
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))


In [ ]:
x_test.shape

## 3. Initializing using a Saved Graph

Here we will construct a new computation graph, but will use a previously trained model to initialize it.

Use the `GraphManager` to load the saved graph and load it into a new session.

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1004)

Construct the forward graph as before, but provide the loaded `graph` as an argument to `__init__`.

In [ ]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

Let `EMI_Driver` know that we already have a session in place.

In [ ]:
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
MODEL_PREFIX

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1039)
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1039)
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1039)
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1039)
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1039)
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)
emiDriver.setSession(sess)

In [ ]:
%%time

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Number of test set samples: %i' % (y_test.shape[0]))

In [ ]:
MODEL_PREFIX

In [ ]:
x_test.shape, bagPredictions.shape,y_test.shape

## 4. Restoring from Numpy Matrices

We first extract the model matrices from the graph and dump it into `.npy` files. Then we load it back again and initialize a new graph with these matrices.

In [ ]:
graph = tf.get_default_graph()
W1 = graph.get_tensor_by_name('W1:0')
B1 = graph.get_tensor_by_name('B1:0')
allVars = emiLSTM.varList + [W1, B1]
sess = emiDriver.getCurrentSession()
allVars = sess.run(allVars)

base = '/tmp/models/'
np.save(base + 'kernel.npy', allVars[0])
np.save(base + 'bias.npy', allVars[1])
np.save(base + 'W1.npy', allVars[2])
np.save(base + 'B1.npy', allVars[3])

Reset the current session and graph

In [ ]:
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

Load the numpy matrices that will be used to initialize the graph.

In [ ]:
base = '/tmp/models/'
kernel = np.load(base + 'kernel.npy')
bias = np.load(base + 'bias.npy')
W = np.load(base + 'W1.npy')
B = np.load(base + 'B1.npy')

Proceed with graph construction as normally done, except that we add the requisite assignment operations.

In [ ]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS,
                                 NUM_OUTPUT)
emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    # Add the assignment operations
    emiLSTM.addBaseAssignOps(graph, [kernel, bias])
    emiLSTM.addExtendedAssignOps(graph, W, B)
    # Setup the driver. You can run the initializations manually as well
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

emiDriver.initializeSession(graph)
# Run the assignment operations
sess = emiDriver.getCurrentSession()
sess.run(emiLSTM.assignOps)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test,
                                                               earlyPolicy_minProb,
                                                               minProb=0.99,
                                                               keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k,
                                             numClass=NUM_OUTPUT)
print('PART IV: Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions ==
                                                        BAG_TEST).astype(int))))